# Predicting NBA Performance Given NCAA Statistics of Basketball Players

The goal of this project is to understand how the performance of a NBA player can be predicted from his performance during NCAA, background and biological features.

This notebook is divided in 3 sections. First, we will understand the data available and transform it such that it is easier to work with. Then we will explore the question of how good a basketball player will be based on NCAA performance. Finally, we will predict which position he will be playing in using the same information.

We import all necessary packages.

In [ ]:
# General packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from math import *
import re
import warnings
from statistics import mode
from scipy.stats import binom

# Sklearn functionalities
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold

## 1. Data Cleaning

There are 5 available dataset with basketball data. We explore each of them separately and determine if they are useful for our analysis.

### 2012-18_officialBoxScore

This dataset has one row for each officer and game for games between seasons 2012-13 and 2017-18. At each row, in addition to the information from the office, there is information about the genral statistics from the two teams that were playing, as well as information about the game.

In [ ]:
official_box_score = pd.read_csv('2012-18_officialBoxScore.csv')
official_box_score.head()

As we are interested in the performance of individual players and not team performance, this dataset is of little interest. Therefore, we will not clean it, even if some data could be encoded in a better way.

### 2012-18_teamBoxScore

This dataframe has two rows for each game that has been played. All of them contain information about the game and performance of both teams. On one of the rows one team is considered thefeatured team and the other one the oponent. The information of this table is the same as the one described before, but organized using a different granularity. Using the same reasons as before, this dataset will not be explored in detail.

In [ ]:
team_box_score = pd.read_csv('2012-18_teamBoxScore.csv')
team_box_score.head()

### 2012-18_standings

This dataset contains information regarding the general standing of teams compared to the rest. As it might be useful to determine the importance of each team, it will be lightly cleaned.

In [ ]:
standings = pd.read_csv('2012-18_standings.csv')
standings.head()


This dataset contains redundant information and messy columns. In order to solve this issues, we will

1. Columns 'stk', 'stkType' and 'sktTot' refer to the strike of wins or losses that a team has. In order to sumarize this information, a single number will describe the strike. It will be positive if it is a winning strike and negative otherwise.

2. Column 'rankOrd' is redundant and should be dropped.

3. Column 'stDate' contains more than one piece of information per cell. That information should be splitted in multiple cells.

To address the latter, we define the following function and create three more columns with the day, month and year of the game.

In [ ]:
# Cleaning function
def cleanDate(df, column, day_name, month_name, year_name):
  '''
  Args:
    df (dataframe): A data frame with a column that contains year, month and day
    column (character): name of the column that contains that information
    day_name (character): name of the column that will contain day information
    day_month (character): name of the column that will contain month information
    day_year (character): name of the column that will contain year information

  Returns:
    None
  '''
  data = pd.to_datetime(df[column])
  df[day_name] = data.dt.day 
  df[month_name] = data.dt.month
  df[year_name] = data.dt.year

# Cleaning the dataset
standings['stk'] = standings['stkTot'] * [-1 if x == 'loss' else 1 for x in standings['stkType']]
standings = standings.drop(columns = ['stkType', 'stkTot', 'rankOrd'])
cleanDate(standings, 'stDate', 'stDay', 'stMonth', 'stYear')
standings.head()

We finally check that there are no missin values in this dataset.

In [ ]:
standings.isna().sum().sum()

### 2012-18_playerBoxScore

This dataset contains the performance of each player at each NBA game for games between seasons 2012-13 and 2017-18, as well as some information regarding the game. This information will be helpful in assesing the performance of different players for their NBA games.

In [ ]:
player_box_score = pd.read_csv('2012-18_playerBoxScore.csv')
player_box_score.head()

Data seems to be relatively clean. We check that each column of the data set containd the data type that should be expected based on the column.

In [ ]:
player_box_score.dtypes

There are some categorical variables that should be better encoded in the dataset. However, as they will not be used in further analysis, there is no need to encode them at this point. Also, sometimes it is easier to keep them this way, as they can make visualizations easier. 

One column that we are going to clean is 'gmDate', as it contains more than one piece of information. 

In [ ]:
# Create new columns for day, month and year
cleanDate(player_box_score, 'gmDate', 'gmDay', 'gmMonth', 'gmYear')
player_box_score.head()

Finally, we check for possible missing values in the columns and we determine the reason behind them.

In [ ]:
player_box_score.isna().sum()

We observe that there are 41 missing values on both columns 'offFNm3' and 'offLNm3'. This columns correspond to the first and last name of the third officer that was present in the game. We check those games to see if they share some peculiarity

In [ ]:
player_box_score[player_box_score['offFNm3'].isna()].head()

Those games do not seem to have anything different from the rest of them. In fact, doing a quick search, it is possible to find that basketball games have between two and three officials. Games whose third official is missing are represented with a missing value on the corresponding cells, such as the one described [here](https://www.basketball-reference.com/boxscores/201503080DET.html).

### college

This dataset contains general statistics of player's performance both in the NBA and in the NCAA, the main US college league. Each row contains the information of one player.

In [ ]:
college = pd.read_csv('college.csv')
college.head()

The first issue with this dataset is that metadata has not been provided and some columns are difficult to interpret. Consequently, we have drafted a description of each column comparing the metrics shown in the dataset to the main metrics for basketball players in the [NBA](https://www.basketball-reference.com) and in the [NCAA](https://www.sports-reference.com/cbb/).
  * Unnamed: 0 - index column.
  * active_from: year a player started  playing in NBA.
  * active_to: year a player finished playing in NBA.
  * birth_date: date of birth of a player.
  * college: college or colleges they attended.
  * height: height of a player in inches-feet.
  * name: first and last name of players.
  * position: position or positions a player uses. The first one if the preferred one if multiple.
  * url: string to add to https://www.basketball-reference.com/ to find data for the NBA stats of the player.
  * weight: weight of the player in pounds.
  
The following columns contain data from the NBA performance of players.
  * NBA__3ptapg: 3-Point Field Goal Attempts Per Game (3PA)
  * NBA__3ptpct: 3-Point Field Goal Percentage (3P%)
  * NBA__3ptpg: 3-Point Field Goal Per Game (3P)
  * NBA__efgpct: Effective Field Goal Percentage (eFG%)
  * NBA_fg%: Field Goal Percentage (FG%)
  * NBA_fg_per_game: Field Goals Per Game (FG)
  * NBA_fga_per_game: Field Goal Attempts Per Game (FGA)
  * NBA_ft%: Free Throw Percentage (FT%)
  * NBA_ft_per_g: Free Throws Per Game (FT)
  * NBA_fta_p_g: Free Throw Attepmts Per Game (FTA)
  * NBA_g_played: Games Played (G)
  * NBA_ppg: Points Per Game (PTS)

The following columns contain data from the NCAA performance of players.
  * NCAA__3ptapg: 3-Point Field Goal Attempts Per Game (3PA)
  * NCAA__3ptpct: 3-Point Field Goal Percentage (3P%)
  * NCAA__3ptpg: 3-Point Field Goal Per Game (3P)
  * NCAA_efgpct: Effective Field Goal Percentage (eFG%)
  *	NCAA_fgapg: Field Goal Attempts Per Game (FGA)
  *	NCAA_fgpct: Field Goal Percentage (FG%)
  *	NCAA_fgpg: Field Goals Per Game (FG)
  *	NCAA_ft: Free Throw Percentage (FT%)
  *	NCAA_ftapg: Free Throw Attepmts Per Game (FTA)
  *	NCAA_ftpg: Free Throws Per Game (FT)
  *	NCAA_games: Games Played (G)
  *	NCAA_ppg: Points Per Game (PTS)

Data from this dataset is very messy. In order to clean it, severall steps are needed. 

1. Drop meaningless columns.

2. Use a better format for the birth date.

3. Encode the universities players attended.

4. Encode height with only one numerical value.

5. Divide first and last name on the name column to ease the comparison of this data with the previous one.

6. Encode the positions for players

7. Address some of the missing values.

8. Ensure that data is in the correct data type.

#### Drop meaningless columns

Columns 'Unnamed: 0' and 'url' are not useful to perform any analysis. Therefore, they should be dropped.

In [ ]:
college = college.drop(columns = ['Unnamed: 0', 'url'], axis = 1)

#### Use a better format for the birth date

The function defined previously is used to encode 'birth_date' in three different columns

In [ ]:
cleanDate(college, 'birth_date', 'birth_day', 'birth_month', 'birth_year')
college = college.drop(columns = 'birth_date')

#### Encode universities

Columns college contains the name of all universities that a player has attended, and a missing value if he has not attended a US college. If he has attended more than one of them , then they are separated by commas. However, this presents an issue when trying to find all different universities that a player has attended, as some universities have a comma in their name, such as "University of California, Los Angeles".

We observe that this peculiarity occurs when there exists an addition to the university name after it. Therefore, a function is created such that, when one of the porions divided by commas does not contain the words University, College or Institute, it is considered to be part of the previous name. This function is applied to create a boolean dataframe that tells if a player has gone to any of the possible universities. If there is a missing value, we consider that he has not attended a US institution, because he has attended a foreign one or because he has not pursued higher eduacation.

In [ ]:
def cleanUniversityLists(college_list):
  '''
  Args:
    college_list (character): character with a list of colleges separated by comma
  Returns:
    list of all colleges
  '''
  college_list = college_list.split(', ')
  trues = np.array([True if re.search('University|College|Institute', x) else False for x in college_list])
  trues = np.append(trues, True)
  return list(np.array([college_list[i] if trues[i+1] else college_list[i]+ ', ' + college_list[i + 1] for i in range(len(trues) - 1)])[trues[:len(trues) - 1]])

colleges = np.unique(college['college'][~college['college'].isna()].apply(cleanUniversityLists).sum())
no_us_college = [False] * len(colleges) + [True]
college_attended = pd.DataFrame([list(pd.Series(colleges).isin(cleanUniversityLists(col))) + [False] if col==col else no_us_college for col in college['college']], 
             columns = np.append(colleges,'No US College'),
             index = college.index)
college_attended.head()

As this dataframe is huge, it might be unnecesary to append it to the original one. To account for the information provided by this dataframe, we will create a feature that reflects if a player has attended a university considered to be one of the best in basketball, as defined in [this site](https://www.ncsasports.org/best-colleges/best-basketball-colleges). We acknowledge that best colleges can change overtime and we might be introducing a source of bias considering those.

In [ ]:
top_colleges = ['University of North Carolina', 
                'University of California, Los Angeles', 
                'Stanford University', 
                'University of Michigan',
                'University of Florida',
                'University of Virginia',
                'Princeton University',
                'Duke University',
                'University of California',
                'Harvard University']
college['Top University'] = (college_attended[top_colleges].sum(axis = 1) > 0).astype('int')
college = college.drop(columns = 'college')

#### Encode height

We find the height in inches of each player

In [ ]:
height = college['height'].str.split('-')
college['height'] = height.str[0].astype('float') + height.str[1].astype('float') * 0.0833333

#### Find first and last name

In order to be consisten with the rest of datasets availables, we split the name into first and last name. If name contains only two words, then this is easy to achieve. If not, it is difficult to know which part of the name is the first or last name. We observe that the number of names with more or less than 2 words is very low

In [ ]:
(college['name'].str.split(' ').str.len() != 2).sum()

Therefore, each of them can be addressed individually. We create two vectors that tracks this singular cases. The first one contains the indices of names with two fisrt names and the second one the ones with one name and multiple last names.

In [ ]:
college['name'][college['name'].str.split(' ').str.len() != 2]

In [ ]:
# First names with two words
name_2 = [232, 638, 1168, 1317, 1876, 2117, 2610, 2939, 3323, 3398, 4332,
         4371, 4405]

# First names with one word
name_1 = [372, 953, 965, 2429, 2608, 4139, 4140, 4141, 4142, 4143, 4144, 
          4145, 4146, 4148, 4179, 4513]

# All of the exceptions to the general rule of naming
names_peculiar = name_2 + name_1

# Vectors that store the first and last names
first_names = college['name'].copy()
last_names = college['name'].copy()

# Assign the first and last name to cases that follow the general norm
mask = first_names.index.isin(names_peculiar)
first_names[~mask] = first_names[~mask].str.split(' ').str[0]
last_names[~mask] = last_names[~mask].str.split(' ').str[1]

# Assign first and last name to peculiar cases
first_names[name_2] = first_names[name_2].str.split(' ').str[0:2].str.join(' ')
last_names[name_2] = last_names[name_2].str.split(' ').str[2:].str.join(' ')
first_names[name_1] = first_names[name_1].str.split(' ').str[0]
last_names[name_1] = last_names[name_1].str.split(' ').str[1:].str.join(' ')

# Incorporate first and last names into the dataframe
college['first_name'] = first_names
college['last_name'] = last_names
college = college.drop(columns = 'name')

#### Encode positions

We encode positions in two different ways. The first one creates three columns that show if a player plays a specific position. The second one determines which is the main position of a player. In basketball, there are 5 positions (https://en.wikipedia.org/wiki/Basketball_positions), but in this dataset they are sumarized in three (F - forward, C - center, G - guard). But, before that, missing values might complicate this procedure. We check for them.

In [ ]:
college[college['position'].isna()]

We observe that there is only one row that does not have a position. As there is only one, and data from almost all row is missing, we drop it.

In [ ]:
college = college.drop(2152)

Now we proceed with the encoding

In [ ]:
# Encode all possible positions
positions = np.unique(college['position'].str.split('-').sum())
position_array = 1 * np.array([college['position'].str.contains(position) for position in positions]).T
college[positions] = pd.DataFrame(position_array, columns = positions, index = college.index)

# Encode main position
college['main_position'] = college['position'].str.split('-').str[0]

# Drop unnecesary columns
college = college.drop(columns = ['position'])

#### Address missing values

There are plenty of missing values on this dataset. We will try to address some of them.

In [ ]:
college.isna().sum()

All values from 'NCAA_efgpct' are missing. However, this value can be found using the variables available as described [here](https://en.wikipedia.org/wiki/Effective_field_goal_percentage).

In [ ]:
college['NCAA_efgpct'] = (college['NCAA_fgpg'] + 0.5 * college['NCAA__3ptpg']) / college['NCAA_fgapg']

Then, three point percentage values are missing if there are no attempts of 3 points. This is reasonable, as the three point percentage is defined as the number of attempts divided by the number of three points.

AWe will assign the missing values due to this reason to 0, as we assume that if a player does not even attempt 3 points, he should not be very good at it. In case this introduces some undesired assumption, we also add a column that indicates that this value was missing.

In [ ]:
# For NBA data
NBA_missing = (college['NBA__3ptapg'] == 0) & (college['NBA__3ptpct'].isna())
college.loc[NBA_missing, 'NBA__3ptpct'] = 0
college['NBA__3ptpct_missing'] = 0
college.loc[NBA_missing, 'NBA__3ptpct_missing'] = 1

# For NCAA values
NCAA_missing = (college['NCAA__3ptapg'] == 0) & (college['NCAA__3ptpct'].isna())
college.loc[NCAA_missing, 'NCAA__3ptpct'] = 0
college['NCAA__3ptpct_missing'] = 0
college.loc[NCAA_missing, 'NCAA__3ptpct_missing'] = 1

There are plenty of columns that have the same ammount of missing values. We check if they are the same

In [ ]:
college[college['NCAA_ppg'].isna()].head()

It looks like all columns that have missing values for 'NCAA_ppg' also have missing values for the rest of the NCAA columns. We check it

In [ ]:
college[college['NCAA_ppg'].isna()].iloc[:, 16:28].isna().sum()

All values except one are missing for the rows that 'NCAA_ppg' are missing. This leads to think that the row that has one value that is not missing is a mistake. As we are interested in predicting the performance of a player based on its university results. If there are no results to base our prediction, it does not make sense to try to do so. Therefore, we create a dataframe with observations that do not have all NCAA data missing.

In [ ]:
college_with_data = college[~college['NCAA_ppg'].isna()]
college_with_data.head()

Having made this midifications, we check the number of missing values remaining

In [ ]:
college_with_data.isna().sum()

There is a remarkable high number of 3 point stats that are missing. Doing a little of research, we find that 3 points were not recorded for old games. This might be the cause why there are many of them missing. As a consequence, we leave them there and address them when we are using different modelling techniques.

The rest of missing values are also left, as they should be modelled with the aim for the data in mind.

#### Data Types

We check that each column has the correct data type

In [ ]:
college_with_data.dtypes

Some of the columns should be integers but are encoded as floats. We code them into integers if possible. However, missing values in some columns make this task impossible, as NaN values cannot be integers.

In [ ]:
college_with_data = college_with_data.astype({'birth_day': 'int',
                                              'birth_month': 'int',
                                              'birth_year': 'int',
                                              'weight': 'int',
                                              'NCAA_games': 'int'})

## 2. Predicting Player Efficiency Rating (PER)

In this section, we will be predicting how good a player will be given their college statistics. Specifically, we will be using the Player Efficiency Rating (PER) to gauge a player's performance. 

Since our datasets do not have the PER of each player, we will have to manually calculate that ourselves using the formula found on this [site](https://www.sportsbettingdime.com/guides/how-to/calculate-per/). Using the player box score dataset, we will first find the cumulative statistics for each player based on all the games they played. Then, we will calculate the PER and normalize it by minutes played.

In [ ]:
# Add up all statistics for each player
player = player_box_score.groupby(['playLNm', 'playFNm']).sum().reset_index()

In [ ]:
# Calculate the player efficiency rate for all players
player['PER'] = (player['playFGM'] * 125.10 + player['playSTL'] * 53.897 + player['play3PM'] * 51.757 + player['playFTM'] * 66.936 
                 + player['playBLK'] * 39.190 + player['playORB'] * 39.190 + player['playAST'] * 34.677 + player['playDRB'] * 14.707
                 - player['playPF'] * 17.174 - player['playFTA'] * 20.091 - player['playFGA'] * 39.190 - player['playTO'] * 53.897) / player['playMin']
player.head()

We notice that some players have extremely high PER since they only have very few minutes played. For instance, this player has a PER of 137.667 having played only played one minute between 2012 and 2018.

In [ ]:
player[player['PER'] == player['PER'].max()]

We will filter out players that played less than 60 minutes in total between 2012 and 2018, since their PER does not likely reflect their true abilities.

In [ ]:
player = player[player['playMin'] >= 60]

Doing a quick sanity check on the distribution of Player Efficiency Rating, we see that it mostly conforms with regards to NBA Standards.

In [ ]:
sns.distplot(player["PER"])
plt.xlabel('Player Efficiency Rating')
plt.ylabel('Percent')
plt.title('Distribution of PER')
plt.show()

The only information we need from this dataset is the PER of each player, so we will drop all other columns except their first and last names.

In [ ]:
player = player[['playLNm', 'playFNm', 'PER']]

We will merge the player's PER score with the college dataset.

In [ ]:
college_nba = player.merge(college_with_data, left_on = ['playLNm', 'playFNm'], right_on = ['last_name', 'first_name'])
college_nba.head()

To enable us to build a stronger model and get more accurate predictions, we will only consider players that are active for at least one year.

In [ ]:
college_nba = college_nba[college_nba['active_from'] != college_nba['active_to']]
college_nba.head()

Since the PER directly correlates with a player's three point shot performance, we will drop those that have null values in 3 point shot in their college data.

In [ ]:
college_nba = college_nba[~college_nba['NCAA__3ptpct'].isna()]
college_nba.head()

Before performing Exploratory Data Analysis, we will separate our dataset into training and testing data by a 80-20 split.

In [ ]:
train, test = train_test_split(college_nba, test_size=0.2, random_state=42)

We are interested in the correlation between a player's performance in NCAA and in NBA. For instance, we look at the field goal percentage of a player in NCAA and NBA and find that they are positively correlated.

In [ ]:
sns.regplot(train["NCAA_fgpct"], train["NBA_fg%"])
plt.xlabel('NCAA Field Goal Percentage')
plt.ylabel('NBA Field Goal Percentage')
plt.title('NCAA vs NBA Field Goal Percentage')
plt.show()

Now, we want to see if the field goal percentage in NCAA has a correlation with the player efficiency rating in NBA.

In [ ]:
sns.regplot(train["NCAA_fgpct"], train["PER"])
plt.xlabel('NCAA Field Goal Percentage')
plt.ylabel('NBA Player Efficiency Rating')
plt.title('NCAA Field Goal Percentage vs NBA Player Efficiency Rating')
plt.show()

We want to find out whether positions affect the PER. It turns out that players in the center position have a higher PER.

In [ ]:
sns.distplot(train[train['main_position'] == 'F']["PER"], label = 'Forward')
sns.distplot(train[train['main_position'] == 'G']["PER"], label = 'Guard')
sns.distplot(train[train['main_position'] == 'C']["PER"], label = 'Center')
plt.xlabel('Player Efficiency Rating')
plt.xlabel('Percent')
plt.title('Distribution of PER Based on Position')
plt.legend()
plt.show()

We want to see which features/columns of our dataset correlate the most with the player efficiency rating. Specifically, we are looking at columns that correspond to a player's biological trait, background, and NCAA performance.

In [ ]:
columns = ['height', 'weight', 'active_from', 'active_to', 'birth_year', 'NCAA_efgpct', 'NCAA_fgapg', 'NCAA__3ptapg', 'NCAA__3ptpct', 'NCAA__3ptpg', 
           'NCAA_fgpct', 'NCAA_fgpg', 'NCAA_ft', 'NCAA_ftapg', 'NCAA_ftpg', 'NCAA_ppg', 'C', 'F', 'G', 'Top University']
correlations = []
for column in columns:
    correlations.append(train[column].corr(train['PER']))
plt.figure(figsize=(12, 8))
plot = sns.barplot(columns, correlations)
plot.set_xticklabels(plot.get_xticklabels(), rotation=45)
plt.xlabel('Features')
plt.ylabel('Correlation with PER')
plt.title('Features and their correlations with PER')
plt.show()

### Ridge Regression

We will now build a ridge regression model that aims to predict PER based on the given features. Since top university and NCAA_fpapg have virtually no correlation with the response variable, they will not be included in the features. We will use the SKlearn pipeline to build our mdoel, and standardize all features using StandardScaler() except the positions (c, f, g) because they were one-hot encoded from the categorical variable.

In [ ]:
ridge_model = Pipeline([
    ("SelectColumns", ColumnTransformer([
        ("Scaler", StandardScaler(), ['height', 'weight', 'active_to', 'active_from', 'birth_year', 'NCAA__3ptapg', 'NCAA__3ptpct', 'NCAA__3ptpg', 
                                      'NCAA_efgpct', 'NCAA_fgpct', 'NCAA_fgpg', 'NCAA_ft', 'NCAA_ftapg', 'NCAA_ftpg', 'NCAA_ppg']),
        ("Keep", "passthrough", ['C', 'F', 'G'])
    ])),
    ("LinearModel", Ridge(alpha = 0.5))
])

In [ ]:
def rmse_score(model, X, y):
    return np.sqrt(np.mean((y - model.predict(X))**2))

In order to find the best hyperparameter alpha, we iterate through 30 equally spaced datapoints between 0 and 15, and find the alpha with the lowest cross validation error.

In [ ]:
alphas = np.linspace(0, 15, 30)
cv_values = []
train_values = []
for alpha in alphas:
    ridge_model.set_params(LinearModel__alpha=alpha)
    ridge_model.fit(train, train['PER'])
    cv_values.append(np.mean(cross_val_score(ridge_model, train, train['PER'], scoring=rmse_score, cv=5)))
    train_values.append(rmse_score(ridge_model, train, train['PER']))

In [ ]:
sns.lineplot(alphas, cv_values)
plt.xlabel('Alpha Values')
plt.ylabel('Cross Validation Error')
plt.title('Cross Validation Error vs Alpha Values')
plt.show()

In [ ]:
sns.lineplot(alphas, train_values)
plt.xlabel('Alpha Values')
plt.ylabel('Training Error')
plt.title('Training Error vs Alpha Values')
plt.show()

After finding the best alpha value, we find the performance of our final ridge regression model.

In [ ]:
best_alpha = alphas[np.argmin(cv_values)]
print("Best Hyperparameter: " + str(best_alpha))
ridge_model.set_params(LinearModel__alpha=best_alpha)
ridge_model.fit(train, train['PER'])
print("Training Score: " + str(rmse_score(ridge_model, train, train['PER'])))
print("Validation Score: " + str(np.mean(cross_val_score(ridge_model, train, train['PER'], scoring = rmse_score, cv=5))))
print("Test Score: " + str(rmse_score(ridge_model, test, test['PER'])))

We compare the performance of our model with the baseline model that always predicts the mean of PER. We find that our model performs much better.

In [ ]:
baseline_model = DummyRegressor("mean")
baseline_model.fit(train, train['PER'])
print("Baseline Score: " + str(rmse_score(baseline_model, train, train['PER'])))

Finally, we graph the residual plot based on our final model. It is a good plot that shows no pattern and a similar vertical spread throughout.

In [ ]:
PER_fitted = ridge_model.predict(train)
residuals = train['PER'] - PER_fitted
residuals.sum() # Sanity check: residuals sum to 0

In [ ]:
plot = sns.scatterplot(PER_fitted, residuals)
plt.xlim((5, 22.5))
plt.xlabel('Fitted PER')
plt.ylabel('Residuals')
plt.title('Residual Plot of Final Model')
plt.show()

## 3. Predicting Position

The aim of this part is to predict which position is a player more likely to fullfill in his profesional life given their college performance.

To do so, we first divide data into training and test before performing exploratory data analysis



In [ ]:
train, test = train_test_split(college_with_data, test_size=0.2, random_state= 55)

We assume that NCAA performance is a good indicator to the position that a player is going to fullfill. We wonder if biological characteristics are also important for this categorization. We start by seeing if height and weight change between categories.

In [ ]:
sns.scatterplot(x = 'height', y = 'weight', data = train[train['main_position'] == 'C'], color = 'red', label = 'C')
sns.scatterplot(x = 'height', y = 'weight', data = train[train['main_position'] == 'F'], color = 'green', label = 'F')
sns.scatterplot(x = 'height', y = 'weight', data = train[train['main_position'] == 'G'], color = 'blue', label = 'G')
plt.title('Height and weight by category')
plt.xlabel('height (in)')
plt.ylabel('weight (lb)')
plt.show()

From the graph it seems that indeed height and weight help determine which position will be assigned to a player.

Other biological features are the day, month and year of birth. We will use the year of birth as a feature to account for modifications in the importance of features as years change. We wonder if the month of birth is important.

In [ ]:
bins = np.arange(.5, 12.6, 1)
sns.distplot(train.loc[train['C'] == 1, 'birth_month'], kde = False, label = 'C', bins = bins, norm_hist = True)
sns.distplot(train.loc[train['F'] == 1, 'birth_month'], kde = False, label = 'F', bins = bins, norm_hist = True)
sns.distplot(train.loc[train['G'] == 1, 'birth_month'], kde = False, label = 'G', bins = bins, norm_hist = True)
plt.legend()
plt.xlabel('Month of brith')
plt.ylabel('Percent per month')
plt.title('Distribution of the month of birth between classes')
plt.show()

We observe that month does not seem to be a good predictor for the position. However, we observe that the number of basketball players born in April is surprisingly small.

In [ ]:
sns.distplot(college_with_data['birth_month'], kde = False, label = 'C', bins = bins, norm_hist = True)
plt.xlabel('Month of brith')
plt.ylabel('Percent per month')
plt.title('Distribution of the month of birth')
plt.show()

It is known that the month of birth of children is a factor that can influence how they perform during early years of school. As kids born in January and kids born in December are relatively different in age but are put in the same class, their development differs. A similar phenomena could happen here. We use hypothesis testing to determine if this difference is due to chance.

We define

* H0: a basketball player is equally likely to be born at any month and any observed difference is due to chance.
* H1: a basketball player is less likely to be born in April than in any other month.

We use a p-value cutoff of 5%.

We use the number of players that are born in April as the test statistic. Small values of the test statistic point towards the alternative hypothesis. The observed value of the test statistic is

In [ ]:
empirical_test_statistic = len(college_with_data[college_with_data['birth_month'] == 4])
empirical_test_statistic

To find the probability of finding a value as extreme as the test statistic, we use a probability distribution.

In [ ]:
distribution = binom(len(college_with_data), 1/12)
sum([distribution.pmf(k) for k in range(empirical_test_statistic + 1)])

As 0.015 < 0.05 reject the null hypothesis in favor of the alternative. However, the alternative hypothesis, that there are less players born in April than any other month, is a side conclusion that sould be explored more deeply. As it is a contraintuitive result, further research should be done. For example, it could be possible that there are less children born in April, which causes that there are less players born that month.

### Predictions

Here, we will predict the position of a player using data from their biological characteristics, as well as the NCAA data. To do so, we first need to addapt the college_with_data dataframe in order to be of use in our techniques.

In [ ]:
college_with_data.head()

In [ ]:
# Columns that contain missing values and will be used for training
# NBA features are removed
missing_columns = train.columns[train.isna().sum() > 0][6:]
# Quantitative columns that will be usedfor training
quantitative_columns = list(train.columns[2:4]) + list(train.columns[16:28]) + ['birth_year']
# Categorical columns that will be used for training
categorical_columns = ['main_position'] + list(train.columns[38:])
# All output columns
#They include columns that will be added posteriorly with information about missing values.
output_columns = quantitative_columns + categorical_columns + [col + '_missing' for col in missing_columns]

# Define and fit scalers and imputers
scaler = StandardScaler()
scaler.fit(train.loc[:, quantitative_columns])

quantitative_imputer = SimpleImputer()
quantitative_imputer.fit(train.loc[:, quantitative_columns])
categorical_imputer = SimpleImputer(strategy = 'most_frequent')
categorical_imputer.fit(train.loc[:, categorical_columns])

def cleanDf(data):
  '''
  Args:
    data (dataframe): data that needs to be transformed
  Returns:
    dataframe with the data with missing values removed, columns explaining 
    where missing values were, quantitative variables scaled and useful 
    columns selected.
  '''
  t = data.copy()

  # Add a binary column that explains where missing values were
  for col in missing_columns:
    t[col + '_missing'] = 0
    t.loc[t[col].isna(), col + '_missing'] = 1

  # Scale and imput columns
  t.loc[:, quantitative_columns] = quantitative_imputer.transform(t.loc[:, quantitative_columns])
  t.loc[:, categorical_columns] = categorical_imputer.transform(t.loc[:, categorical_columns])
  t.loc[:, quantitative_columns] = scaler.transform(t.loc[:, quantitative_columns])
  
  # Only return output_columns
  return t.loc[:, output_columns]
  

We find the training and test cleaned data

In [ ]:
clean_train = cleanDf(train)
clean_test = cleanDf(test)
X_train = clean_train.drop(columns = 'main_position')
Y_train = clean_train['main_position']
X_test = clean_test.drop(columns = 'main_position')
Y_test = clean_test['main_position']

#### Logistic Regression

Now we fit a multi-class logistic regression to try to predict the main position of players. We will use cross validation to determine which norm penalty or parameters are the best for this model. We also add a line such that the warnings are ignored.

The reason behind this is that, the only solver that allows l1 penalties and multiple classes in logistic regression does not converge. However, we have tested the 'newton-cg' solver when the penalty is l2 and the errors in both models are equivalent. Therefore, we consider that using 'saga' does not hurt our predictions, but we will use 'newton-cg' when possible.

In [ ]:
def accuracy(model, X, y):
  '''
  Args:
    model: an sklearn model that accepts a predict function
    X (dataframe): features to predict from
    y (list-like): correct predictions for data in X
  Returns:
    accuracy of the model
  '''
  return np.mean(model.predict(X) == y)

warnings.filterwarnings("ignore")
errors = {}
for norm, c in [(norm, c) for norm in ['l1', 'l2'] for c in np.arange(1, 50, 5)]:
  logistic_model = LogisticRegression(penalty = norm, C = c, solver = 'saga', random_state = 55)
  errors[norm + ' c=' + str(c)] = np.mean(cross_val_score(logistic_model, X_train, Y_train, scoring = accuracy, cv = 5))

max(errors.keys(), key = lambda x: errors[x]), max(errors.values())

We will try different models before commiting to the best one. But first, we wish to explore which features are better to predict each of the outputs. 

To do so, we use the preoperties of the l1 norm as feature selection. We start with a very high penalty and then gradually reduce it. We will see which coefficients stop being zero first, as they will be the more important when predicting.

In [ ]:
# Find the coefficients for the logistic regression models
coeffs_C = list()
coeffs_F = list()
coeffs_G = list()
for c in 10.0 ** np.arange(-3, 5, .1):
  logistic_model = LogisticRegression(penalty = 'l1', C = c, solver = 'saga', random_state = 42)
  logistic_model.fit(X_train, Y_train)
  coeffs_C.append(logistic_model.coef_[0])
  coeffs_F.append(logistic_model.coef_[1])
  coeffs_G.append(logistic_model.coef_[2])

In [ ]:
# The best coefficients to predict centers are
coeffs_C = pd.DataFrame(coeffs_C, columns = X_train.columns)
coeffs_C['C'] = 10.0 ** np.arange(-3, 5, .1)
coeffs_C_plot = coeffs_C.loc[:, coeffs_C.ne(0).idxmax().sort_values()[:10].index]
coeffs_C_plot.plot(x = 'C', logx = True).legend(loc='center left',bbox_to_anchor=(1.0, 0.5))
plt.ylabel('Coefficients')
plt.title('Change in coefficients for LASSO regression and class C')
plt.show()

In [ ]:
# The best coefficients to predict fronts are
coeffs_F = pd.DataFrame(coeffs_F, columns = X_train.columns)
coeffs_F['C'] = 10.0 ** np.arange(-3, 5, .1)
coeffs_F_plot = coeffs_F.loc[:, coeffs_F.ne(0).idxmax().sort_values()[:10].index]
coeffs_F_plot.plot(x = 'C', logx = True).legend(loc='center left',bbox_to_anchor=(1.0, 0.5))
plt.ylabel('Coefficients')
plt.title('Change in coefficients for LASSO regression and class F')
plt.show()

In [ ]:
# The best coefficients to predict guards are
coeffs_G = pd.DataFrame(coeffs_G, columns = X_train.columns)
coeffs_G['C'] = 10.0 ** np.arange(-3, 5, .1)
coeffs_G_plot = coeffs_G.loc[:, coeffs_G.ne(0).idxmax().sort_values()[:10].index]
coeffs_G_plot.plot(x = 'C', logx = True).legend(loc='center left',bbox_to_anchor=(1.0, 0.5))
plt.title('Change in coefficients for LASSO regression and class G')
plt.ylabel('Coefficients')
plt.show()

As it was expected, height and weight have a great impact over the prediction. Also, some measure of time, such as the year of birth or the active_to columns appear on all three predictions. This leads to think that what is important for one position has changed overtime. 

Finally, when we are predicting fronts, we see that some lines are overlapped. This is a consequence that some columns have missing values at the exact same position in the training data. However, as we don't know how the test set is going to behave, we leave all columns to make the model more robust.

#### PCA Regression


As we have seen that some features are represented multiple times on the dataset, we can try PCA. If we find a representation of data in a lower dimension, we won't have repeated features. We define a PCA function and use as predictors the PCA components. We use cross validation to determine the best number of predictors. We have restricted the values that we try for the sake of time execution, but more values have been tested.

In [ ]:
# Parameters for PCA
u, s, vt = np.linalg.svd(X_train, full_matrices = False)
X_mean = np.mean(X_train)
X_sd = np.std(X_train)

def PCA(t, k):
  '''
  Args:
    t (dataframe): data to perform PCA to
    k: number of principal components
  Returns:
    dataframe of k principal components of t
  '''
  D = (t - X_mean) / X_sd
  Z = D @ vt[:k, :].T
  Z.columns = ['PC' + str(k) for k in range(1, k+1)]
  return Z

errors_PCA = {}
for norm, c, k in [(norm, c, k) for norm in ['l1', 'l2'] 
                   for c in np.arange(35, 46, 5) 
                   for k in np.arange(15, len(X_train.columns) + 1)]:
  PCA_logistic_model = LogisticRegression(penalty = norm, C = c, solver = 'saga', random_state = 55)
  errors_PCA[norm + ' c=' + str(c) + ' k=' + str(k)] = np.mean(
      cross_val_score(PCA_logistic_model, PCA(X_train, k), 
                      Y_train, scoring = accuracy, cv = 5))
  
max(errors_PCA.keys(), key = lambda x: errors_PCA[x]), max(errors_PCA.values())

#### Random Forest

We observe that there has not been an improvement over linear regression. We explore why this might be. We plot data using the first two principal components and divide it by category.

In [ ]:
sns.scatterplot(data = PCA(X_train, 2), x = 'PC1', y = 'PC2', hue = Y_train)
plt.legend()
plt.title('First principal components of observations by position')
plt.show()

From the PCA plot, it seems that the positions might not have linear boundaries between them. As logistic regression imposes linear boundaries, it might be useful to try a method that does not, such as random forests.

In [ ]:
forest_model = RandomForestClassifier(n_estimators = 50, random_state = 55)
np.mean(
      cross_val_score(forest_model, X_train, 
                      Y_train, scoring = accuracy, cv = 5))

#### Majority vote model

As the errors produed by all three models are very similar, we try a majority vote, to see if the limitations of one can be solved by the other two. We will use the best parameters for each model. We implement cross validation to check is this majority vote model performs better.

In [ ]:
# Define all models with best parameters
logistic_model = LogisticRegression(penalty = 'l1', C = 6, solver = 'saga', random_state = 55)
PCA_logistic_model = LogisticRegression(penalty = 'l1', C = 40, solver = 'saga', random_state = 55)
forest_model = RandomForestClassifier(n_estimators = 50, random_state = 55)

# Perform cross-validation
folds = KFold(n_splits = 5)
error = []
for tr, val in folds.split(X_train):
  X_train_cv = X_train.iloc[tr, :]
  Y_train_cv = Y_train.iloc[tr]
  X_val = X_train.iloc[val, :]
  Y_val = Y_train.iloc[val]
  logistic_model.fit(X_train_cv, Y_train_cv)
  PCA_logistic_model.fit(PCA(X_train_cv, 17), Y_train_cv)
  forest_model.fit(X_train_cv, Y_train_cv)
  logistic_y = logistic_model.predict(X_val)
  PCA_y = PCA_logistic_model.predict(PCA(X_val, 17))
  forest_y = forest_model.predict(X_val)
  y_predicted = [mode([logistic_y[i], PCA_y[i], forest_y[i]]) for i in range(len(logistic_y))]
  error.append(np.mean(y_predicted == Y_val))
np.mean(error)

We observe that this cross-validation error is sligtly smaller than the one obtained from logistic regression and logistic regression with PCA. However, the differece is very small and it could be easily due to chance. As we believe our last model to be more robust, we will use it for predictions.

In [ ]:
def predictPosition(X):
  '''
  Args:
    X (dataframe): features for prediction
  Returns:
    list with predictions
  '''
  logistic_y = logistic_model.predict(X)
  PCA_y = PCA_logistic_model.predict(PCA(X, 17))
  forest_y = forest_model.predict(X)
  y_predicted = [mode([logistic_y[i], PCA_y[i], forest_y[i]]) for i in range(len(logistic_y))]
  return y_predicted

np.mean(predictPosition(X_test) == Y_test)

We have found a classifier that performs with an accuracy of 89% approximately.

### Performance by class

We study the distribution of classes over the test set to see if there is a major imbalance, i.e., some class is being badly predicted.

In [ ]:
# Predictions
Y_hat = np.array(predictPosition(X_test))
# Real classes
Y_test = np.array(Y_test)

# Classification matrix
class_matrix = pd.DataFrame([[sum((Y_hat == i) & (Y_test == j)) for i in ['C', 'F', 'G']] 
                             for j in ['C', 'F', 'G']], 
             columns = ['Predicted C', 'Predicted F', 'Predicted G'], 
             index = ['Actual C', 'Actual F', 'Actual G'])
class_matrix

We observe that all classes are represented more or less correctly in this table. Maybe a great proportion of centers are misclassified as forwards. As there are less centers in the dataset, the penalty for misclassifying them is smaller than for other classes.